<a href="https://colab.research.google.com/github/rodrigo-fortes/TCC_PUCMG_CD/blob/main/TCC_PUCMG_CUSTEIO_RURAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pontifícia Universidade Católica de Minas Gerais - PUC-MG**
# Especialização em Ciência da Dados e Big Data
# Códigos Referentes ao Trabalho de Conclusão de Curso

## **ANÁLISE DE PREVISIBILIDADE DE CUSTEIO RURAL POR PRODUTO PARA MUNICÍPIOS DO ESTADO DE MINAS GERAIS**

## Aluno: Rodrigo Fortes Lopes

.
.

**Fontes dos dados:**


Bacen - Matriz de Dados do Crédito Rural - MDCR
Dados agregados relativos a operações de crédito rural registradas no sistema Sicor, contratadas por produtores rurais em instituições financeiras.

https://dadosabertos.bcb.gov.br/dataset/matrizdadoscreditorural

https://dadosabertos.bcb.gov.br/dataset/matrizdadoscreditorural/resource/5e70f0f7-61ea-4c39-b7ed-2dae9166bdca

.
.

DTB - Divisão Territorial Brasileira

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/23701-divisao-territorial-brasileira.html

.
.

Códigos dos municípios IBGE

https://www.ibge.gov.br/explica/codigos-dos-municipios.php

.
.

Áreas Territoriais

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15761-areas-dos-municipios.html


## Conexões e pacotes texto em itálico

In [ ]:
# Acesso ao Google Drive que contém o material de estudo

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Pacotes para análises e gráficos
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

## Carga dos dados e Identificação das Colunas




In [ ]:
# Arquivos CSV

custeio = pd.read_csv('/content/drive/MyDrive/PUC Minas - Ciência de Dados e Big Data/17 - TCC - CIÊNCIA DE DADOS E BIG DATA/Credito_Rural/05_Contratos_de_Custeio_por_Municipio_e_Produto/Custeio.csv',
                 sep = ',', encoding = 'utf-8',engine='c')

ibge = pd.read_csv('/content/drive/MyDrive/PUC Minas - Ciência de Dados e Big Data/17 - TCC - CIÊNCIA DE DADOS E BIG DATA/Credito_Rural/21_Microrregioes_IBGE/RELATORIO_DTB_BRASIL_MUNICIPIO_v2.csv',
                 sep = ',', encoding = 'utf-8',engine='c')

area = pd.read_csv('/content/drive/MyDrive/PUC Minas - Ciência de Dados e Big Data/17 - TCC - CIÊNCIA DE DADOS E BIG DATA/Credito_Rural/21_Microrregioes_IBGE/area_municipios.csv',
                 sep = ',', encoding = 'utf-8',engine='c')

In [ ]:
# Colunas de custeio
custeio.info(verbose=True, null_counts=True)

In [ ]:
# Primeiras linhas da base custeio
custeio.head()

In [ ]:
# Colunas de ibge
ibge.info()

In [ ]:
# Primeiras linhas da base igbe
ibge.head()

In [ ]:
# Colunas de area
area.info()

In [ ]:
# Primeiras linhas da base area
area.head()

## Ajustes dos dados

In [ ]:
# Trocar "," por "." na coluna de Valor de Custeio

custeio['VlCusteio'] = custeio['VlCusteio'].replace({',': '.'}, regex=True)

In [ ]:
# Converte para número o AnoEmissao, MesEmissao, VlCusteio

custeio['AnoEmissao'] = pd.to_numeric(custeio['AnoEmissao'], errors='coerce')
custeio['MesEmissao'] = pd.to_numeric(custeio['MesEmissao'], errors='coerce')
custeio['VlCusteio'] = pd.to_numeric(custeio['VlCusteio'], errors='coerce')

In [ ]:
# Modifica os nomes das colunas de Códigos Municipais para que fiquem iguais e permitam fazer o merge das tabelas

ibge.rename(columns={'Código Município Completo': 'codIbge'}, inplace = True)
area.rename(columns={'CD_MUN': 'codIbge'}, inplace = True)

In [ ]:
# Ajustes no campo de área municipal

area['AR_MUN_2022'] = area['AR_MUN_2022'].replace({',': '.'}, regex=True)
area['AR_MUN_2022'] = pd.to_numeric(area['AR_MUN_2022'], errors='coerce')

## Análise e Exploração dos Dados

In [ ]:
# Cópia da Base

custeio_ae = custeio.copy()
ibge_ae = ibge.copy()
area_ae = area.copy()

In [ ]:
custeio_ae.head()

In [ ]:
len(custeio_ae['nomeProduto'].unique())

In [ ]:
custeio_ae.info(verbose=True, null_counts=True)

In [ ]:
custeio_ae = custeio_ae[custeio_ae.cdEstado.isin([12])]

In [ ]:
custeio_ae.info()

In [ ]:
len(custeio_ae['nomeProduto'].unique())

In [ ]:
# Atividade (contagem)

fig, ax = plt.subplots(figsize=(12,5))
sns.countplot(data=custeio_ae, x="AnoEmissao", hue="Atividade")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.barplot(data=custeio_ae, x="AnoEmissao", y="VlCusteio", hue="Atividade", estimator=sum)

ax.set_xlabel("Ano de Emissão")
ax.set_ylabel("Valor de Custeio")
ax.set_title("Soma do Valor de Custeio por Ano e Atividade")

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.scatterplot(data=custeio_ae, x = 'AnoEmissao', y = 'VlCusteio')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.boxplot(data=custeio_ae, x = 'AnoEmissao', y = 'VlCusteio')

In [ ]:
a = custeio_ae['nomeProduto'].value_counts(normalize=True)
a = pd.DataFrame(a)
a.head(11)

In [ ]:
# Filtrar os "nomeProduto" que existem em todos os "AnoEmissao"
produto_anos = custeio_ae.groupby('nomeProduto')['AnoEmissao'].nunique()
nome_produto_todos_anos = produto_anos[produto_anos == produto_anos.max()].index

# Filtrar os registros apenas para os "nomeProduto" desejados
data_filtrada = custeio_ae[custeio_ae['nomeProduto'].isin(nome_produto_todos_anos)]

# Calcular a soma total de "VlCusteio" para cada "nomeProduto"
soma_custeio_por_produto = data_filtrada.groupby('nomeProduto')['VlCusteio'].sum()

# Identificar os 10 "nomeProduto" com a maior soma de "VlCusteio"
top_10_produtos = soma_custeio_por_produto.nlargest(10)

print("Os 10 maiores nomeProduto em soma de VlCusteio:")
print(top_10_produtos)



In [ ]:
# Seleção dos 10 produtos com maior soma de VlCusteio e que aparecem em todos os anos da série
selec10 = ['"BOVINOS"', '"CAFÉ"', '"SOJA"', '"MILHO"', '"CANA-DE-AÇUCAR"',
           '"SUÍNOS"', '"ALHO"', '"BATATA-INGLESA"', '"FEIJÃO"', '"BANANA"']

# Calcular a soma total de "VlCusteio" para cada "nomeProduto"
soma_custeio_por_produto = custeio_ae.groupby('nomeProduto')['VlCusteio'].sum().reset_index()

# Filtrar os produtos desejados
df_soma = soma_custeio_por_produto[soma_custeio_por_produto['nomeProduto'].isin(selec10)]

# Ordenar em ordem decrescente de soma de VlCusteio
df_soma = df_soma.sort_values('VlCusteio', ascending=False)

# Criar um gráfico de barras para visualizar a soma de VlCusteio por nomeProduto (apenas os 10 maiores)
plt.barh(df_soma['nomeProduto'], df_soma['VlCusteio'])
plt.xlabel('Soma de VlCusteio')
plt.ylabel('nomeProduto')
plt.title('Soma de VlCusteio por nomeProduto (10 maiores)')
plt.show()


In [ ]:
# Merge com IBGE
ibge_ae.rename(columns={'Código Município Completo': 'codIbge'}, inplace = True)

custeio_meso = pd.merge(custeio_ae, ibge_ae, how = 'left')
custeio_meso.head()

In [ ]:
custeio_meso['Nome_Mesorregião'].unique()

In [ ]:
len(custeio_meso['Nome_Mesorregião'].unique())

In [ ]:
# Calcular a soma de VlCusteio por mesorregião
df_sum_mesorregiao = custeio_meso.groupby('Nome_Mesorregião')['VlCusteio'].sum()

plt.rcParams.update({'font.size': 18})

# Calcular a participação percentual
df_percentual = df_sum_mesorregiao / df_sum_mesorregiao.sum() * 100

# Ordenar os valores em ordem decrescente
df_percentual = df_percentual.sort_values(ascending=False)

# Gerar um gráfico de pizza
plt.figure(figsize=(10, 10))
df_percentual.plot(kind='pie', autopct='%1.1f%%')
plt.ylabel('')
plt.title('Participação Percentual de VlCusteio por Mesorregião')
plt.show()

In [ ]:
# Calcular a soma de VlCusteio por Mesorregião e nomeProduto
df_grouped = custeio_meso.groupby(['Nome_Mesorregião', 'nomeProduto'])['VlCusteio'].sum().reset_index()

top10_nomeProduto_por_mesorregiao = (df_grouped.groupby('Nome_Mesorregião').apply(lambda x: x.nlargest(10,
                                                                    'VlCusteio')).reset_index(drop=True))

# Criar um gráfico para cada Mesorregião
for mesorregiao in top10_nomeProduto_por_mesorregiao['Nome_Mesorregião'].unique():
    df_mesorregiao = top10_nomeProduto_por_mesorregiao[top10_nomeProduto_por_mesorregiao['Nome_Mesorregião'] == mesorregiao]
    plt.figure(figsize=(10, 6))  # Ajuste o tamanho do gráfico conforme necessário
    ax = sns.barplot(data=df_mesorregiao, y='nomeProduto', x='VlCusteio', color='skyblue', orient='horizontal')
    plt.xlabel('Soma de VlCusteio')
    plt.ylabel('Nome do Produto')
    plt.title(f'Soma de VlCusteio por nomeProduto - Mesorregião: {mesorregiao}')
    plt.tight_layout()  # Para melhorar a visualização dos rótulos
    plt.show()




In [ ]:
# Agrupar por nomeProduto e Nome_Mesorregião e calcular a média de VlCusteio por Município por Ano
df_grouped = custeio_meso.groupby(['nomeProduto','AnoEmissao', 'Municipio','Nome_Mesorregião'])['VlCusteio'].sum().reset_index()
df_grouped = df_grouped.groupby(['nomeProduto','Nome_Mesorregião'])['VlCusteio'].mean().reset_index()

# Obter a lista de nomeProduto
produtos = df_grouped['nomeProduto'].unique()

# Criar um gráfico para cada nomeProduto
for produto in produtos:
    # Filtrar os dados para o nomeProduto atual
    df_produto = df_grouped[df_grouped['nomeProduto'] == produto]

    # Ordenar os dados por soma de VlCusteio em ordem decrescente
    df_produto = df_produto.sort_values(by='VlCusteio', ascending=False)

    # Criar o gráfico de barras horizontais
    plt.figure(figsize=(10, 6))
    plt.barh(df_produto['Nome_Mesorregião'], df_produto['VlCusteio'], color='skyblue')

    # Configurar os rótulos e título do gráfico
    plt.xlabel('Média de Valor de Custeio')
    plt.ylabel('Nome da Mesorregião')
    plt.title(f'Média de VlCusteio por Nome_Mesorregião - nomeProduto: {produto}')

    # Inverter a ordem do eixo y para exibir as barras na ordem correta
    plt.gca().invert_yaxis()

    # Exibir o gráfico
    plt.tight_layout()
    plt.show()

In [ ]:
area_ae.head()

In [ ]:
# Histograma das áreas municipais de MG

# Filtrar os dados para CD_UF = 12
dados_filtrados = area_ae[area_ae["CD_UF"] == 12]

# Gerar o histograma
plt.hist(dados_filtrados["AR_MUN_2022"], bins=20)
plt.xlabel("Área Municipal (2022)")
plt.ylabel("Frequência")
plt.title("Histograma da Área Municipal (2022) para CD_UF = 12")
plt.show()


In [ ]:
# Calcular os quartis
quartis = dados_filtrados["AR_MUN_2022"].quantile([0.25, 0.5, 0.75])

print("Q1:", quartis[0.25])
print("Q2 (Mediana):", quartis[0.5])
print("Q3:", quartis[0.75])


In [ ]:
# Merge com area_ae

custeio_meso2 = pd.merge(custeio_meso, area_ae, how = 'left')
custeio_meso3 = custeio_meso2[['Municipio', 'Nome_Mesorregião', 'nomeProduto',
                               'MesEmissao', 'AnoEmissao', 'cdEstado', 'VlCusteio', 'AR_MUN_2022']]
custeio_meso3.head()

In [ ]:
custeio_meso3.info()

In [ ]:
# FAIXAS

# Função Trimestral

def func_trimestre_01(x):
  if x == 1:
    return "primeiro"
  elif x == 2:
    return "primeiro"
  elif x == 3:
    return "primeiro"
  elif x == 4:
    return "segundo"
  elif x == 5:
    return "segundo"
  elif x == 6:
    return "segundo"
  elif x == 7:
    return "terceiro"
  elif x == 8:
    return "terceiro"
  elif x == 9:
    return "terceiro"
  else:
    return "quarto"

# Função para semestre hidrológico

def func_sh_01(x):
  if x == 1:
    return "chuva"
  elif x == 2:
    return "chuva"
  elif x == 3:
    return "chuva"
  elif x == 4:
    return "seca"
  elif x == 5:
    return "seca"
  elif x == 6:
    return "seca"
  elif x == 7:
    return "seca"
  elif x == 8:
    return "seca"
  elif x == 9:
    return "seca"
  else:
    return "chuva"

# Função para Áreas Municipais

def func_area_01(x):
  if x <= 2700:
    return 1
  elif x <= 5350:
    return 2
  elif x <= 8630:
    return 3
  else:
    return 4

# Funções para Faixas de Custeio

# Mensal

def func_faixa_m_01(x):
  if x <= 150000:
    return 1
  elif x <= 2000000:
    return 10
  else:
    return 20

# Trimestral

def func_faixa_t_01(x):
  if x <= 500000:
    return 1
  elif x <= 5000000:
    return 10
  else:
    return 20

# Semestral

def func_faixa_s_01(x):
  if x <= 1000000:
    return 1
  elif x <= 10000000:
    return 10
  else:
    return 20

# Anual

def func_faixa_a_01(x):
  if x <= 2000000:
    return 1
  elif x <= 20000000:
    return 10
  else:
    return 20


In [ ]:
# Faixas
custeio_meso3['AR_MUN_2022'] = custeio_meso3['AR_MUN_2022'].apply(func_area_01)
custeio_meso3['TRIMESTRE'] = custeio_meso3['MesEmissao'].apply(func_trimestre_01)
custeio_meso3['SEMESTRE'] = custeio_meso3['MesEmissao'].apply(func_sh_01)

custeio_meso3.head()

In [ ]:
nomeProduto = ['"MILHO"', '"BOVINOS"', '"CAFÉ"', '"SOJA"', '"SUÍNOS"', '"AVICULTURA"',
           '"ALHO"',  '"MORANGO"','"OVINOS"','"PLANTAS ORNAMENTAIS"',
           '"EUCALIPTO"', '"CANA-DE-AÇUCAR"','"TANGERINA"','"TOMATE"','"TRIGO"','"UVA"']

# Filtrar os dados para o estado desejado (cdEstado = 12) e selecionar apenas as colunas relevantes
dados_filtrados = custeio_meso3[(custeio_meso3["cdEstado"] == 12)][["Nome_Mesorregião", "nomeProduto", "AnoEmissao", "VlCusteio"]]

# Loop pelos nomeProduto para gerar os gráficos
for produto in nomeProduto:
    # Filtrar os dados pelo nomeProduto atual
    dados_produto = dados_filtrados[dados_filtrados["nomeProduto"] == produto]

    # Calcular o valor mediano de VlCusteio agrupado por ano e Nome_Mesorregião
    median_values = dados_produto.groupby(["AnoEmissao", "Nome_Mesorregião"])["VlCusteio"].median().reset_index()

    # Plotar o gráfico de barras horizontais
    plt.figure(figsize=(10, 6))
    sns.barplot(data=median_values, x="VlCusteio", y="Nome_Mesorregião", hue="AnoEmissao", orient="h")
    plt.xlabel("Valor Mediano de VlCusteio")
    plt.ylabel("Nome_Mesorregião")
    plt.title(f"Gráfico de Barras Horizontais - {produto}")
    plt.legend(title="AnoEmissao")
    plt.show()


In [ ]:
nomeProduto = ['"MILHO"', '"BOVINOS"', '"CAFÉ"', '"SOJA"', '"SUÍNOS"', '"AVICULTURA"',
           '"ALHO"',  '"MORANGO"','"OVINOS"','"PLANTAS ORNAMENTAIS"',
           '"EUCALIPTO"', '"CANA-DE-AÇUCAR"','"TANGERINA"','"TOMATE"','"TRIGO"','"UVA"']

# Filtrar os dados para o estado desejado (cdEstado = 12) e selecionar apenas as colunas relevantes
dados_filtrados = custeio_meso3[(custeio_meso3["cdEstado"] == 12)][["Nome_Mesorregião", "nomeProduto", "SEMESTRE", "VlCusteio"]]

# Loop pelos nomeProduto para gerar os gráficos
for produto in nomeProduto:
    # Filtrar os dados pelo nomeProduto atual
    dados_produto = dados_filtrados[dados_filtrados["nomeProduto"] == produto]

    # Calcular o valor mediano de VlCusteio agrupado por SEMESTRE e Nome_Mesorregião
    median_values = dados_produto.groupby(["SEMESTRE", "Nome_Mesorregião"])["VlCusteio"].median().reset_index()

    # Plotar o gráfico de barras horizontais
    plt.figure(figsize=(10, 6))
    sns.barplot(data=median_values, x="VlCusteio", y="Nome_Mesorregião", hue="SEMESTRE", orient="h")
    plt.xlabel("Valor Mediano de VlCusteio")
    plt.ylabel("Nome_Mesorregião")
    plt.title(f"Gráfico de Barras Horizontais - {produto}")
    plt.legend(title="SEMESTRE")
    plt.show()


In [ ]:
nomeProduto = ['"MILHO"', '"BOVINOS"', '"CAFÉ"', '"SOJA"', '"SUÍNOS"', '"AVICULTURA"',
           '"ALHO"',  '"MORANGO"','"OVINOS"','"PLANTAS ORNAMENTAIS"',
           '"EUCALIPTO"', '"CANA-DE-AÇUCAR"','"TANGERINA"','"TOMATE"','"TRIGO"','"UVA"']

# Filtrar os dados para o estado desejado (cdEstado = 12) e selecionar apenas as colunas relevantes
dados_filtrados = custeio_meso3[(custeio_meso3["cdEstado"] == 12)][["Nome_Mesorregião", "nomeProduto", "TRIMESTRE", "VlCusteio"]]

# Loop pelos nomeProduto para gerar os gráficos
for produto in nomeProduto:
    # Filtrar os dados pelo nomeProduto atual
    dados_produto = dados_filtrados[dados_filtrados["nomeProduto"] == produto]

    # Calcular o valor mediano de VlCusteio agrupado por TRIMESTRE e Nome_Mesorregião
    median_values = dados_produto.groupby(["TRIMESTRE", "Nome_Mesorregião"])["VlCusteio"].median().reset_index()

    # Plotar o gráfico de barras horizontais
    plt.figure(figsize=(10, 6))
    sns.barplot(data=median_values, x="VlCusteio", y="Nome_Mesorregião", hue="TRIMESTRE", orient="h")
    plt.xlabel("Valor Mediano de VlCusteio")
    plt.ylabel("Nome_Mesorregião")
    plt.title(f"Gráfico de Barras Horizontais - {produto}")
    plt.legend(title="TRIMESTRE")
    plt.show()


In [ ]:
nomeProduto = ['"MILHO"', '"BOVINOS"', '"CAFÉ"', '"SOJA"', '"SUÍNOS"', '"AVICULTURA"',
           '"ALHO"',  '"MORANGO"','"OVINOS"','"PLANTAS ORNAMENTAIS"',
           '"EUCALIPTO"', '"CANA-DE-AÇUCAR"','"TANGERINA"','"TOMATE"','"TRIGO"','"UVA"']

# Filtrar os dados para o estado desejado (cdEstado = 12) e selecionar apenas as colunas relevantes
dados_filtrados = custeio_meso3[(custeio_meso3["cdEstado"] == 12)][["Nome_Mesorregião", "nomeProduto", "MesEmissao", "VlCusteio"]]

# Loop pelos nomeProduto para gerar os gráficos
for produto in nomeProduto:
    # Filtrar os dados pelo nomeProduto atual
    dados_produto = dados_filtrados[dados_filtrados["nomeProduto"] == produto]

    # Calcular o valor mediano de VlCusteio agrupado por MesEmissao e Nome_Mesorregião
    median_values = dados_produto.groupby(["MesEmissao", "Nome_Mesorregião"])["VlCusteio"].median().reset_index()

    # Plotar o gráfico de barras horizontais
    plt.figure(figsize=(10, 6))
    sns.barplot(data=median_values, x="VlCusteio", y="Nome_Mesorregião", hue="MesEmissao", orient="h")
    plt.xlabel("Valor Mediano de VlCusteio")
    plt.ylabel("Nome_Mesorregião")
    plt.title(f"Gráfico de Barras Horizontais - {produto}")
    plt.legend(title="MesEmissao")
    plt.show()


In [ ]:
nomeProduto = ['"MILHO"', '"BOVINOS"', '"CAFÉ"', '"SOJA"', '"SUÍNOS"', '"AVICULTURA"',
           '"ALHO"',  '"MORANGO"','"OVINOS"','"PLANTAS ORNAMENTAIS"',
           '"EUCALIPTO"', '"CANA-DE-AÇUCAR"','"TANGERINA"','"TOMATE"','"TRIGO"','"UVA"']

# Filtrar os dados para cada nomeProduto e selecionar apenas as colunas relevantes
for produto in nomeProduto:
    dados_produto = custeio_meso3[custeio_meso3["nomeProduto"] == produto][["AR_MUN_2022", "AnoEmissao", "VlCusteio"]]

    # Calcular o valor mediano de VlCusteio agrupado por AR_MUN_2022 e AnoEmissao
    median_values = dados_produto.groupby(["AR_MUN_2022", "AnoEmissao"])["VlCusteio"].median().reset_index()

    # Plotar o gráfico de barras horizontais
    plt.figure(figsize=(10, 6))
    sns.barplot(data=median_values, x="VlCusteio", y="AR_MUN_2022", hue="AnoEmissao", orient="h")
    plt.xlabel("Valor Mediano de VlCusteio")
    plt.ylabel("AR_MUN_2022")
    plt.title(f"Gráfico de Barras Horizontais - {produto}")
    plt.legend(title="AnoEmissao")
    plt.show()


In [ ]:
# Lista de nomeProduto
selecao = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']

# Filtrar o dataframe original pela seleção de nomeProduto
df_selecao = custeio_meso[custeio_meso['nomeProduto'].isin(selecao)]

# Calcular a média anual de VlCusteio por nomeProduto e Nome_Mesorregião
df_mediana_anual = df_selecao.groupby(['nomeProduto', 'Municipio','AnoEmissao','Nome_Mesorregião'])['VlCusteio'].median().reset_index()
df_media_anual = df_mediana_anual.groupby(['nomeProduto','Nome_Mesorregião'])['VlCusteio'].mean().reset_index()

# Criar tabela pivot usando pivot_table()
df_pivot = pd.pivot_table(df_media_anual, index='nomeProduto', columns='Nome_Mesorregião', values='VlCusteio')

# Transpor a tabela (inverter linhas e colunas)
df_pivot = df_pivot.transpose()

# Formatar a tabela com duas casas decimais e separador de milhar
df_pivot = df_pivot.round(2).applymap('{:,.0f}'.format)

# Exibir a tabela resultante
print(df_pivot)


In [ ]:
# Lista de nomeProduto
selecao = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']

# Filtrar o dataframe original pela seleção de nomeProduto
df_selecao = custeio_meso[custeio_meso['nomeProduto'].isin(selecao)]

# Calcular a média anual de VlCusteio por nomeProduto e Nome_Mesorregião
df_mediana_anual = df_selecao.groupby(['nomeProduto', 'Municipio','AnoEmissao','Nome_Mesorregião'])['VlCusteio'].sum().reset_index()
df_media_anual = df_mediana_anual.groupby(['nomeProduto','Nome_Mesorregião'])['VlCusteio'].mean().reset_index()

# Criar tabela pivot usando pivot_table()
df_pivot = pd.pivot_table(df_media_anual, index='nomeProduto', columns='Nome_Mesorregião', values='VlCusteio')

# Transpor a tabela (inverter linhas e colunas)
df_pivot = df_pivot.transpose()

# Formatar a tabela com duas casas decimais e separador de milhar
df_pivot = df_pivot.round(2).applymap('{:,.0f}'.format)

# Exibir a tabela resultante
print(df_pivot)

In [ ]:
# Agrupar por nomeProduto, Nome_Mesorregião e Microrregião e calcular a média de VlCusteio por Município por Ano
df_grouped2 = custeio_meso.groupby(['nomeProduto','AnoEmissao', 'Municipio','Nome_Mesorregião',
                                    'Nome_Microrregião'])['VlCusteio'].sum().reset_index()
df_grouped2 = df_grouped2.groupby(['nomeProduto','Nome_Mesorregião','Nome_Microrregião'])['VlCusteio'].mean().reset_index()
df_grouped2

In [ ]:
meso = ['Zona da Mata']
df_grouped3 = df_grouped2[df_grouped2['Nome_Mesorregião'].isin(meso)]

# Agrupar os dados por microrregião e somar os valores de VlCusteio para cada produto
df_sum_vlcusteio = df_grouped3.groupby(['Nome_Microrregião', 'nomeProduto'])['VlCusteio'].sum()

# Obter os cinco principais produtos por microrregião
df_top_products = df_sum_vlcusteio.groupby('Nome_Microrregião').nlargest(5).reset_index(level=1, drop=True).reset_index()

# Iterar sobre os grupos e gerar os gráficos
for microrregiao, df_top in df_top_products.groupby('Nome_Microrregião'):
    plt.figure()
    plt.figure(figsize=(10, 6))
    plt.barh(df_top['nomeProduto'], df_top['VlCusteio'])
    plt.xlabel('VlCusteio')
    plt.ylabel('Nome do Produto')
    plt.title(f'5 Principais Produtos - Microrregião: {microrregiao}')
    plt.tight_layout()

plt.show()

## Modelo para previsão em 3 (três) faixas de valores

### Criação das funções de definição de faixas

In [ ]:
# Função Trimestral

def func_trimestre(x):
  if x == 1:
    return "primeiro"
  elif x == 2:
    return "primeiro"
  elif x == 3:
    return "primeiro"
  elif x == 4:
    return "segundo"
  elif x == 5:
    return "segundo"
  elif x == 6:
    return "segundo"
  elif x == 7:
    return "terceiro"
  elif x == 8:
    return "terceiro"
  elif x == 9:
    return "terceiro"
  else:
    return "quarto"

In [ ]:
# Funções para Faixas de Custeio

# Trimestral

def func_faixa_t(x):
  if x <= 125000:
    return 1
  elif x <= 750000:
    return 10
  else:
    return 15


# Anual

def func_faixa_a(x):
  if x <= 600000:
    return 1
  elif x <= 3000000:
    return 5
  else:
    return 15



### Modelo Trimestral

In [ ]:
# Preparação das bases

custeio_tf = custeio.copy()
ibge_tf = ibge.copy()
area_tf = area.copy()

In [ ]:
# Função trimestral

custeio_tf['Trimestre'] = custeio_tf['MesEmissao'].apply(func_trimestre)
custeio_tf

In [ ]:
# Soma de VlCusteio por município por trimestre

custeio_tf_s = custeio_tf.groupby(['Municipio', 'nomeProduto', 'AnoEmissao','Trimestre', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()
custeio_tf_s

In [ ]:
# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio

custeio_tf_mg = custeio_tf_s[custeio_tf_s['cdEstado'] == 12]
valores_desejados = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']
custeio_tf_mg_prod = custeio_tf_mg[custeio_tf_mg['nomeProduto'].isin(valores_desejados)]
anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_tf_mg_prod = custeio_tf_mg_prod[custeio_tf_mg_prod['AnoEmissao'].isin(anos)]

custeio_tf_mg_prod.head()

In [ ]:
# Merge das bases de dados

custeio_tf_mg_prod_ibge = pd.merge(custeio_tf_mg_prod, ibge_tf, how = 'left')
custeio_tf_mg_prod_ibge_area = pd.merge(custeio_tf_mg_prod_ibge, area_tf, how = 'left')

In [ ]:
# Seleção das colunas

custeio_tf_mg_prod_ibge_area_2 = custeio_tf_mg_prod_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'AnoEmissao', 'Trimestre', 'AR_MUN_2022','VlCusteio']]
custeio_tf_mg_prod_ibge_area_2

In [ ]:
# Utilização das faixas de valores de custeio

custeio_tf_mg_prod_ibge_area_2['VlCusteio'] = custeio_tf_mg_prod_ibge_area_2['VlCusteio'].apply(func_faixa_t)
custeio_tf_mg_prod_ibge_area_2

In [ ]:
# Gera a lista de variáveis categóricas
cols_cat_tf = custeio_tf_mg_prod_ibge_area_2.select_dtypes(include='object').columns.tolist()


cols_cat_tf

In [ ]:
# Criação dos dummies
data_tf = pd.get_dummies(custeio_tf_mg_prod_ibge_area_2, columns=cols_cat_tf, drop_first=True)

data_tf.head()

In [ ]:
# Definição das variáveis dependentes e independentes

X, y = data_tf.drop(['VlCusteio'], axis=1), data_tf['VlCusteio']
class_names = data_tf['VlCusteio']

# Particiona a base de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

In [ ]:
# Árvore de Decisão

from sklearn.tree import DecisionTreeClassifier
tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
tree_custo = tree_custo.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
y_pred = tree_custo.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, y_pred))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb = nb.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", nb.score(X_train, y_train))
tp_nb = nb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_nb))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_nb)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando Gradiente Descendente

from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd = sgd.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", sgd.score(X_train, y_train))
tp_sgd = sgd.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_sgd))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_sgd)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando KNN (K - Nearest Neighbors)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn = knn.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", knn.score(X_train, y_train))
tp_knn = knn.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_knn))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_knn)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando Randon Forest

from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm = rfm.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", rfm.score(X_train, y_train))
tp_rfm = rfm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_rfm))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_rfm)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Modelo SVC

from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", svm.score(X_train, y_train))
tp_svm = svm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_svm))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_svm)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", gb.score(X_train, y_train))
tp_gb = gb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_gb))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_gb)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# AdaBoostClassifier

from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", ada.score(X_train, y_train))
tp_ada = ada.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_ada))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_ada)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier()
et.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", et.score(X_train, y_train))
tp_et = et.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_et))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_et)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# LogisticRegression

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", lr.score(X_train, y_train))
tp_lr = lr.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_lr))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_lr)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Armazena os resultados em um dicionário
resultados = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for modelo, resultado in resultados.items():
    df_resultados = pd.DataFrame.from_dict(resultado, orient='index', columns=['Valor'])
    df_resultados.index.name = modelo
    print(df_resultados)
    print('\n')


### Modelo Trimestral (Sem 'AnoEmissao')

In [ ]:
# Preparação das bases

custeio_tf = custeio.copy()
ibge_tf = ibge.copy()
area_tf = area.copy()

In [ ]:
# Função trimestral

custeio_tf['Trimestre'] = custeio_tf['MesEmissao'].apply(func_trimestre)
custeio_tf

In [ ]:
# Soma de VlCusteio por município por trimestre

custeio_tf_s = custeio_tf.groupby(['Municipio', 'nomeProduto', 'AnoEmissao','Trimestre', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()
custeio_tf_s

In [ ]:
# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio

custeio_tf_mg = custeio_tf_s[custeio_tf_s['cdEstado'] == 12]
valores_desejados = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']
custeio_tf_mg_prod = custeio_tf_mg[custeio_tf_mg['nomeProduto'].isin(valores_desejados)]
anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_tf_mg_prod = custeio_tf_mg_prod[custeio_tf_mg_prod['AnoEmissao'].isin(anos)]

custeio_tf_mg_prod.head()

In [ ]:
# Merge das bases de dados

custeio_tf_mg_prod_ibge = pd.merge(custeio_tf_mg_prod, ibge_tf, how = 'left')
custeio_tf_mg_prod_ibge_area = pd.merge(custeio_tf_mg_prod_ibge, area_tf, how = 'left')

In [ ]:
# Seleção das colunas

custeio_tf_mg_prod_ibge_area_2 = custeio_tf_mg_prod_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'Trimestre', 'AR_MUN_2022','VlCusteio']]
custeio_tf_mg_prod_ibge_area_2

In [ ]:
# Utilização das faixas de valores de custeio

custeio_tf_mg_prod_ibge_area_2['VlCusteio'] = custeio_tf_mg_prod_ibge_area_2['VlCusteio'].apply(func_faixa_t)
custeio_tf_mg_prod_ibge_area_2

In [ ]:
# Gera a lista de variáveis categóricas
cols_cat_tf = custeio_tf_mg_prod_ibge_area_2.select_dtypes(include='object').columns.tolist()


cols_cat_tf

In [ ]:
# Criação dos dummies
data_tf = pd.get_dummies(custeio_tf_mg_prod_ibge_area_2, columns=cols_cat_tf, drop_first=True)

data_tf.head()

In [ ]:
# Definição das variáveis dependentes e independentes

X, y = data_tf.drop(['VlCusteio'], axis=1), data_tf['VlCusteio']
class_names = data_tf['VlCusteio']

# Particiona a base de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

In [ ]:
# Árvore de Decisão

from sklearn.tree import DecisionTreeClassifier
tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
tree_custo = tree_custo.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
y_pred = tree_custo.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, y_pred))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb = nb.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", nb.score(X_train, y_train))
tp_nb = nb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_nb))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_nb)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando Gradiente Descendente

from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd = sgd.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", sgd.score(X_train, y_train))
tp_sgd = sgd.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_sgd))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_sgd)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando KNN (K - Nearest Neighbors)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn = knn.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", knn.score(X_train, y_train))
tp_knn = knn.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_knn))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_knn)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando Randon Forest

from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm = rfm.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", rfm.score(X_train, y_train))
tp_rfm = rfm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_rfm))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_rfm)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_tf['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Modelo SVC

from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", svm.score(X_train, y_train))
tp_svm = svm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_svm))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_svm)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", gb.score(X_train, y_train))
tp_gb = gb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_gb))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_gb)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# AdaBoostClassifier

from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", ada.score(X_train, y_train))
tp_ada = ada.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_ada))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_ada)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier()
et.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", et.score(X_train, y_train))
tp_et = et.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_et))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_et)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# LogisticRegression

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", lr.score(X_train, y_train))
tp_lr = lr.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_lr))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_lr)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_tf['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_tf['VlCusteio'].unique()])
print(cnf_table)



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Armazena os resultados em um dicionário
resultados = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for modelo, resultado in resultados.items():
    df_resultados = pd.DataFrame.from_dict(resultado, orient='index', columns=['Valor'])
    df_resultados.index.name = modelo
    print(df_resultados)
    print('\n')


In [ ]:
import numpy as np
# Dados de acurácia para treino e teste
modelos = ['Árvore de Decisão', 'Naive Bayes', 'Gradiente Descendente', 'KNN',
           'Random Forest', 'SVC', 'GradientBoostingClassifier', 'AdaBoostClassifier',
           'ExtraTreesClassifier', 'Regressão Logística']

acuracia_treino = [0.768697, 0.407952, 0.490372, 0.746676, 0.768670, 0.488324,
                   0.608032, 0.574920, 0.768697, 0.558298]

acuracia_teste = [0.672331, 0.407053, 0.485400, 0.655178, 0.671374, 0.481809,
                  0.594862, 0.564465, 0.671374, 0.549067]

# Configuração do gráfico
fig, ax = plt.subplots(figsize=(10, 6))
indice = np.arange(len(modelos))
largura = 0.35

# Barras de acurácia para treino e teste
rects1 = ax.barh(indice - largura/2, acuracia_treino, largura, label='Treinamento')
rects2 = ax.barh(indice + largura/2, acuracia_teste, largura, label='Teste')

# Rótulos, título e legenda
ax.set_yticks(indice)
ax.set_yticklabels(modelos)
ax.set_xlabel('Acurácia')
ax.set_title('Acurácia de Treinamento e Teste por Modelo')
ax.legend()

# Adiciona rótulos nas barras
def autolabel(rects):
    for rect in rects:
        width = rect.get_width()
        ax.annotate(f'{width:.2f}', xy=(width, rect.get_y() + rect.get_height() / 2),
                    xytext=(3, 0), textcoords="offset points",
                    ha='left', va='center')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

### Modelo Anual

In [ ]:
# Preparação das bases

custeio_af = custeio.copy()
ibge_af = ibge.copy()
area_af = area.copy()

In [ ]:
# Soma de VlCusteio por município por ano

custeio_af_s = custeio_af.groupby(['Municipio', 'nomeProduto', 'AnoEmissao', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()
custeio_af_s

In [ ]:
# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio

custeio_af_mg = custeio_af_s[custeio_af_s['cdEstado'] == 12]
valores_desejados = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']
custeio_af_mg_prod = custeio_af_mg[custeio_af_mg['nomeProduto'].isin(valores_desejados)]

anos = [2019, 2020, 2021, 2022]#[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_af_mg_prod = custeio_af_mg_prod[custeio_af_mg_prod['AnoEmissao'].isin(anos)]

custeio_af_mg_prod.head()

In [ ]:
# Merge das bases de dados

custeio_af_mg_prod_ibge = pd.merge(custeio_af_mg_prod, ibge_af, how = 'left')
custeio_af_mg_prod_ibge_area = pd.merge(custeio_af_mg_prod_ibge, area_af, how = 'left')

In [ ]:
# Seleção das colunas

custeio_af_mg_prod_ibge_area_2 = custeio_af_mg_prod_ibge_area[['nomeProduto',	'Nome_Mesorregião','AR_MUN_2022','VlCusteio']]
custeio_af_mg_prod_ibge_area_2

In [ ]:
# Utilização das faixas de valores de custeio

custeio_af_mg_prod_ibge_area_2['VlCusteio'] = custeio_af_mg_prod_ibge_area_2['VlCusteio'].apply(func_faixa_a)
custeio_af_mg_prod_ibge_area_2

In [ ]:
# Gera a lista de variáveis categóricas
cols_cat_af = custeio_af_mg_prod_ibge_area_2.select_dtypes(include='object').columns.tolist()

cols_cat_af

In [ ]:
# Cria Dummies
data_af = pd.get_dummies(custeio_af_mg_prod_ibge_area_2, columns=cols_cat_af, drop_first=True)

data_af.head()

In [ ]:
# Definição das variáveis dependentes e independentes

X, y = data_af.drop(['VlCusteio'], axis=1), data_af['VlCusteio']
class_names = data_af['VlCusteio']

# Particiona a base de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

In [ ]:
# Árvore de Decisão

from sklearn.tree import DecisionTreeClassifier
tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
tree_custo = tree_custo.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
y_pred = tree_custo.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, y_pred))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_af['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:

from sklearn.tree import export_graphviz
import graphviz

# Treinamento do modelo
tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
tree_custo = tree_custo.fit(X_train, y_train)

# Converter os valores das classes para strings
class_names = tree_custo.classes_.astype(str).tolist()

# Gerar a figura da árvore de decisão
dot_data = export_graphviz(tree_custo, out_file=None, feature_names=X_train.columns.tolist(), class_names=class_names, filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)

# Exibir a figura da árvore de decisão
graph.render("decision_tree")  # Salvar a figura como arquivo "decision_tree.pdf"
graph.view()  # Abrir visualização interativa da árvore de decisão


In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb = nb.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", nb.score(X_train, y_train))
tp_nb = nb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_nb))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_nb)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_af['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando Gradiente Descendente

from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd = sgd.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", sgd.score(X_train, y_train))
tp_sgd = sgd.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_sgd))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_sgd)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_af['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando KNN (K - Nearest Neighbors)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn = knn.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", knn.score(X_train, y_train))
tp_knn = knn.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_knn))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_knn)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_af['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
#Criação do modelo utilizando Randon Forest

from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm = rfm.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", rfm.score(X_train, y_train))
tp_rfm = rfm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_rfm))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_rfm)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_af['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)

In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", svm.score(X_train, y_train))
tp_svm = svm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_svm))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_svm)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_af['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", gb.score(X_train, y_train))
tp_gb = gb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_gb))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_gb)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_af['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", ada.score(X_train, y_train))
tp_ada = ada.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_ada))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_ada)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_af['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier()
et.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", et.score(X_train, y_train))
tp_et = et.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_et))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_et)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_af['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)


In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

print("-----------------------------------------------------------------")
print("Dados de Acurácia")
print("")
print("Acurácia: ", lr.score(X_train, y_train))
tp_lr = lr.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))

print("-----------------------------------------------------------------")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_lr))

print("-----------------------------------------------------------------")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_lr)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_af['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_af['VlCusteio'].unique()])
print(cnf_table)



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Armazena os resultados em um dicionário
resultados = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for modelo, resultado in resultados.items():
    df_resultados = pd.DataFrame.from_dict(resultado, orient='index', columns=['Valor'])
    df_resultados.index.name = modelo
    print(df_resultados)
    print('\n')



In [ ]:

import numpy as np

# Dados de acurácia para treino e teste
modelos = ['Árvore de Decisão', 'Naive Bayes', 'Gradiente Descendente', 'KNN',
           'Random Forest', 'SVC', 'GradientBoostingClassifier', 'AdaBoostClassifier',
           'ExtraTreesClassifier', 'Regressão Logística']

acuracia_treino = [0.861322, 0.444149, 0.485182, 0.766147, 0.861322, 0.486322,
                   0.653875, 0.598594, 0.861322, 0.584536]

acuracia_teste = [0.729345, 0.435328, 0.482051, 0.605698, 0.736752, 0.488319,
                  0.645014, 0.609687, 0.733903, 0.583476]

# Configuração do gráfico
fig, ax = plt.subplots(figsize=(10, 6))  # Ajusta o tamanho da figura
indice = np.arange(len(modelos))
largura = 0.5  # Ajusta a largura das barras

# Barras de acurácia para treino e teste
rects1 = ax.barh(indice - largura/2, acuracia_treino, largura, label='Treinamento', color='blue')
rects2 = ax.barh(indice + largura/2, acuracia_teste, largura, label='Teste', color='orange')

# Rótulos, título e legenda
ax.set_yticks(indice)
ax.set_yticklabels(modelos)
ax.set_xlabel('Acurácia')
ax.set_title('Acurácia de Treinamento e Teste por Modelo')
ax.legend()

# Adiciona rótulos nas barras
def autolabel(rects):
    for rect in rects:
        width = rect.get_width()
        ax.annotate(f'{width:.2f}', xy=(width, rect.get_y() + rect.get_height() / 2),
                    xytext=(3, 0), textcoords="offset points",
                    ha='left', va='center')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()


## Modelos Mesorregionais Anual e Trimestral Compactos

### Trimestral

In [ ]:
# Função Trimestral
def func_trimestre(x):
  if x == 1:
    return "primeiro"
  elif x == 2:
    return "primeiro"
  elif x == 3:
    return "primeiro"
  elif x == 4:
    return "segundo"
  elif x == 5:
    return "segundo"
  elif x == 6:
    return "segundo"
  elif x == 7:
    return "terceiro"
  elif x == 8:
    return "terceiro"
  elif x == 9:
    return "terceiro"
  else:
    return "quarto"


# Função para Áreas Municipais
def func_area(x):
  if x <= 2700:
    return 'Faixa 1'
  elif x <= 5350:
    return 'Faixa 2'
  elif x <= 8630:
    return 'Faixa 3'
  else:
    return 'Faixa 4'

# Funções para Faixas de Custeio
# Trimestral
def func_faixa_t(x):
  if x <= 125000:
    return 1
  elif x <= 750000:
    return 10
  else:
    return 15


# Preparação das bases
custeio_mt = custeio.copy()
ibge_mt = ibge.copy()
area_mt = area.copy()

# Aplicação da funcão trimestral
custeio_mt['Trimestre'] = custeio_mt['MesEmissao'].apply(func_trimestre)

# Soma de VlCusteio por município por trimestre
custeio_mt_s = custeio_mt.groupby(['Municipio', 'nomeProduto', 'AnoEmissao','Trimestre', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()

# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio
custeio_mt_mg = custeio_mt_s[custeio_mt_s['cdEstado'] == 12]
valores_desejados = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']
custeio_mt_mg_prod = custeio_mt_mg[custeio_mt_mg['nomeProduto'].isin(valores_desejados)]
anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_mt_mg_prod = custeio_mt_mg_prod[custeio_mt_mg_prod['AnoEmissao'].isin(anos)]

# Merge das bases de dados
custeio_mt_mg_prod_ibge = pd.merge(custeio_mt_mg_prod, ibge_mt, how = 'left')
custeio_mt_mg_prod_ibge_area = pd.merge(custeio_mt_mg_prod_ibge, area_mt, how = 'left')

# Seleção das colunas
custeio_mt_mg_prod_ibge_area_2 = custeio_mt_mg_prod_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'Nome_Microrregião', 'Trimestre','AR_MUN_2022','VlCusteio']]

# Utilização das faixas de valores de custeio
custeio_mt_mg_prod_ibge_area_2['VlCusteio'] = custeio_mt_mg_prod_ibge_area_2['VlCusteio'].apply(func_faixa_t)

# Identificação dos nomes das mesorregiões
meso = custeio_mt_mg_prod_ibge_area_2['Nome_Mesorregião'].unique()

# Criação de um dicionário para armazenar os resultados
resultados = {}

for nome in meso:
  print('--------------------------------------------------------------------------------')
  print(f'Análise da Mesorregiao: ',nome)

  # Escolha da mesorregião
  custeio_mt_mg_prod_ibge_area_3 = custeio_mt_mg_prod_ibge_area_2[custeio_mt_mg_prod_ibge_area_2['Nome_Mesorregião'] == nome]

  # Identificação das Microrregiões de cada Mesorregião
  micro = custeio_mt_mg_prod_ibge_area_3['Nome_Microrregião'].unique()
  print(f'Essa Mesorregiao é formada por {len(micro)} Microrregiões: ',micro)

  # Selecionar colunas
  custeio_mt_mg_prod_ibge_area_3 = custeio_mt_mg_prod_ibge_area_3[['nomeProduto', 'Nome_Microrregião', 'Trimestre','AR_MUN_2022','VlCusteio']]

  # Gera a lista de variáveis categóricas
  cols_cat_mt = custeio_mt_mg_prod_ibge_area_3.select_dtypes(include='object').columns.tolist()

  # Criação das colunas de Dummies
  data_mt = pd.get_dummies(custeio_mt_mg_prod_ibge_area_3, columns=cols_cat_mt, drop_first=True)

  # Definição das variáveis dependentes e independentes
  X, y = data_mt.drop(['VlCusteio'], axis=1), data_mt['VlCusteio']
  class_names = data_mt['VlCusteio']

  # Particiona a base de dados
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

  # Árvore de Decisão
  from sklearn.tree import DecisionTreeClassifier
  tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
  tree_custo = tree_custo.fit(X_train, y_train)
  print("")
  print("Modelo de Árvore de Decisão - DecisionTreeClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
  y_pred = tree_custo.predict(X_test)
  print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, y_pred))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, y_pred)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Naive Bayes
  from sklearn.naive_bayes import GaussianNB
  nb = GaussianNB()
  b = nb.fit(X_train, y_train)
  print("")
  print("Modelo de Naive Bayes - GaussianNB")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", nb.score(X_train, y_train))
  tp_nb = nb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_nb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_nb)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Gradiente Descendente
  from sklearn.linear_model import SGDClassifier
  sgd = SGDClassifier()
  sgd = sgd.fit(X_train, y_train)
  print("")
  print("Modelo Gradiente Descendente - SGDClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", sgd.score(X_train, y_train))
  tp_sgd = sgd.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_sgd))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_sgd)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando KNN (K - Nearest Neighbors)
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier()
  knn = knn.fit(X_train, y_train)
  print("")
  print("Modelo KNN (K - Nearest Neighbors) - KNeighborsClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", knn.score(X_train, y_train))
  tp_knn = knn.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_knn))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_knn)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Randon Forest
  from sklearn.ensemble import RandomForestClassifier
  rfm = RandomForestClassifier()
  rfm = rfm.fit(X_train, y_train)
  print("")
  print("Modelo Randon Forest - RandomForestClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", rfm.score(X_train, y_train))
  tp_rfm = rfm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_rfm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_rfm)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo SVC
  from sklearn.svm import SVC
  svm = SVC()
  svm.fit(X_train, y_train)
  print("")
  print("Modelo SVC - Support Vector Classification")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", svm.score(X_train, y_train))
  tp_svm = svm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_svm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_svm)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # GradientBoostingClassifier
  from sklearn.ensemble import GradientBoostingClassifier
  gb = GradientBoostingClassifier()
  gb.fit(X_train, y_train)
  print("")
  print("Modelo GradientBoostingClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", gb.score(X_train, y_train))
  tp_gb = gb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_gb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_gb)
  nf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo AdaBoostClassifier
  from sklearn.ensemble import AdaBoostClassifier
  ada = AdaBoostClassifier()
  ada.fit(X_train, y_train)
  print("")
  print("Modelo AdaBoostClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", ada.score(X_train, y_train))
  tp_ada = ada.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_ada))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_ada)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo ExtraTreesClassifier
  from sklearn.ensemble import ExtraTreesClassifier
  et = ExtraTreesClassifier()
  et.fit(X_train, y_train)
  print("")
  print("Modelo ExtraTreesClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", et.score(X_train, y_train))
  tp_et = et.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_et))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_et)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Regressão Logística
  from sklearn.linear_model import LogisticRegression
  lr = LogisticRegression()
  lr.fit(X_train, y_train)
  print("")
  print("Modelo Regressão Logística")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", lr.score(X_train, y_train))
  tp_lr = lr.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_lr))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_lr)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Armazena os resultados em um dicionário
  resultados[nome] = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for nome, resultados_mesorregiao in resultados.items():
    print(f'Análise da Mesorregião: {nome}\n')
    df_resultados = pd.DataFrame.from_dict(resultados_mesorregiao, orient='index')
    print(df_resultados)
    print('\n')



### Anual

In [ ]:
# Função para Áreas Municipais
def func_area(x):
  if x <= 2700:
    return 'Faixa 1'
  elif x <= 5350:
    return 'Faixa 2'
  elif x <= 8630:
    return 'Faixa 3'
  else:
    return 'Faixa 4'

# Funções para Faixas de Custeio
# Anual
def func_faixa_a(x):
  if x <= 600000:
    return 1
  elif x <= 3000000:
    return 5
  else:
    return 15


# Preparação das bases
custeio_ma = custeio.copy()
ibge_ma = ibge.copy()
area_ma = area.copy()

# Soma de VlCusteio por município por ano
custeio_ma_s = custeio_ma.groupby(['Municipio', 'nomeProduto', 'AnoEmissao', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()

# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio
custeio_ma_mg = custeio_ma_s[custeio_ma_s['cdEstado'] == 12]
valores_desejados = ['"BOVINOS"','"CAFÉ"', '"MILHO"', '"SOJA"']
custeio_ma_mg_prod = custeio_ma_mg[custeio_ma_mg['nomeProduto'].isin(valores_desejados)]
anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_ma_mg_prod = custeio_ma_mg_prod[custeio_ma_mg_prod['AnoEmissao'].isin(anos)]

# Merge das bases de dados
custeio_ma_mg_prod_ibge = pd.merge(custeio_ma_mg_prod, ibge_ma, how = 'left')
custeio_ma_mg_prod_ibge_area = pd.merge(custeio_ma_mg_prod_ibge, area_ma, how = 'left')

# Seleção das colunas
custeio_ma_mg_prod_ibge_area_2 = custeio_ma_mg_prod_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'Nome_Microrregião','AR_MUN_2022','VlCusteio']]

# Utilização das faixas de valores de custeio
custeio_ma_mg_prod_ibge_area_2['VlCusteio'] = custeio_ma_mg_prod_ibge_area_2['VlCusteio'].apply(func_faixa_a)

# Identificação dos nomes das mesorregiões
meso = custeio_ma_mg_prod_ibge_area_2['Nome_Mesorregião'].unique()

for nome in meso:
  print('--------------------------------------------------------------------------------')
  print(f'Análise da Mesorregiao: ',nome)

  # Zona da Mata
  custeio_ma_mg_prod_ibge_area_3 = custeio_ma_mg_prod_ibge_area_2[custeio_ma_mg_prod_ibge_area_2['Nome_Mesorregião'] == nome]

  # Identificação das Microrregiões de cada Mesorregião
  micro = custeio_ma_mg_prod_ibge_area_3['Nome_Microrregião'].unique()
  print(f'Essa Mesorregiao é formada por {len(micro)} Microrregiões: ',micro)

  # Selecionar colunas
  custeio_ma_mg_prod_ibge_area_3 = custeio_ma_mg_prod_ibge_area_3[['nomeProduto', 'Nome_Microrregião','AR_MUN_2022','VlCusteio']]

  # Gera a lista de variáveis categóricas
  cols_cat_ma = custeio_ma_mg_prod_ibge_area_3.select_dtypes(include='object').columns.tolist()

  # Criação das colunas de Dummies
  data_ma = pd.get_dummies(custeio_ma_mg_prod_ibge_area_3, columns=cols_cat_ma, drop_first=True)

  # Definição das variáveis dependentes e independentes
  X, y = data_ma.drop(['VlCusteio'], axis=1), data_ma['VlCusteio']
  class_names = data_ma['VlCusteio']

  # Particiona a base de dados
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

  # Árvore de Decisão
  from sklearn.tree import DecisionTreeClassifier
  tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
  tree_custo = tree_custo.fit(X_train, y_train)
  print("")
  print("Modelo de Árvore de Decisão - DecisionTreeClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
  y_pred = tree_custo.predict(X_test)
  print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, y_pred))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, y_pred)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Naive Bayes
  from sklearn.naive_bayes import GaussianNB
  nb = GaussianNB()
  b = nb.fit(X_train, y_train)
  print("")
  print("Modelo de Naive Bayes - GaussianNB")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", nb.score(X_train, y_train))
  tp_nb = nb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_nb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_nb)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Gradiente Descendente
  from sklearn.linear_model import SGDClassifier
  sgd = SGDClassifier()
  sgd = sgd.fit(X_train, y_train)
  print("")
  print("Modelo Gradiente Descendente - SGDClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", sgd.score(X_train, y_train))
  tp_sgd = sgd.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_sgd))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_sgd)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando KNN (K - Nearest Neighbors)
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier()
  knn = knn.fit(X_train, y_train)
  print("")
  print("Modelo KNN (K - Nearest Neighbors) - KNeighborsClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", knn.score(X_train, y_train))
  tp_knn = knn.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_knn))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_knn)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Randon Forest
  from sklearn.ensemble import RandomForestClassifier
  rfm = RandomForestClassifier()
  rfm = rfm.fit(X_train, y_train)
  print("")
  print("Modelo Randon Forest - RandomForestClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", rfm.score(X_train, y_train))
  tp_rfm = rfm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_rfm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_rfm)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo SVC
  from sklearn.svm import SVC
  svm = SVC()
  svm.fit(X_train, y_train)
  print("")
  print("Modelo SVC - Support Vector Classification")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", svm.score(X_train, y_train))
  tp_svm = svm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_svm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_svm)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # GradientBoostingClassifier
  from sklearn.ensemble import GradientBoostingClassifier
  gb = GradientBoostingClassifier()
  gb.fit(X_train, y_train)
  print("")
  print("Modelo GradientBoostingClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", gb.score(X_train, y_train))
  tp_gb = gb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_gb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_gb)
  nf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo AdaBoostClassifier
  from sklearn.ensemble import AdaBoostClassifier
  ada = AdaBoostClassifier()
  ada.fit(X_train, y_train)
  print("")
  print("Modelo AdaBoostClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", ada.score(X_train, y_train))
  tp_ada = ada.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_ada))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_ada)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo ExtraTreesClassifier
  from sklearn.ensemble import ExtraTreesClassifier
  et = ExtraTreesClassifier()
  et.fit(X_train, y_train)
  print("")
  print("Modelo ExtraTreesClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", et.score(X_train, y_train))
  tp_et = et.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_et))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_et)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Regressão Logística
  from sklearn.linear_model import LogisticRegression
  lr = LogisticRegression()
  lr.fit(X_train, y_train)
  print("")
  print("Modelo ExtraTreesClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", lr.score(X_train, y_train))
  tp_lr = lr.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_lr))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_lr)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Armazena os resultados em um dicionário
  resultados[nome] = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for nome, resultados_mesorregiao in resultados.items():
    print(f'Análise da Mesorregião: {nome}\n')
    df_resultados = pd.DataFrame.from_dict(resultados_mesorregiao, orient='index')
    print(df_resultados)
    print('\n')



### Trimestral com Seleção de Produtos

In [ ]:
# Função Trimestral
def func_trimestre(x):
  if x == 1:
    return "primeiro"
  elif x == 2:
    return "primeiro"
  elif x == 3:
    return "primeiro"
  elif x == 4:
    return "segundo"
  elif x == 5:
    return "segundo"
  elif x == 6:
    return "segundo"
  elif x == 7:
    return "terceiro"
  elif x == 8:
    return "terceiro"
  elif x == 9:
    return "terceiro"
  else:
    return "quarto"


# Funções para Faixas de Custeio
# Trimestral
def func_faixa_t(x):
  if x <= 125000:
    return 1
  elif x <= 750000:
    return 10
  else:
    return 15


# Preparação das bases
custeio_mt = custeio.copy()
ibge_mt = ibge.copy()
area_mt = area.copy()

# Aplicação da funcão trimestral
custeio_mt['Trimestre'] = custeio_mt['MesEmissao'].apply(func_trimestre)

# Soma de VlCusteio por município por trimestre
custeio_mt_s = custeio_mt.groupby(['Municipio', 'nomeProduto', 'AnoEmissao','Trimestre', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()

# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio
custeio_mt_mg = custeio_mt_s[custeio_mt_s['cdEstado'] == 12]

anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_mt_mg = custeio_mt_mg[custeio_mt_mg['AnoEmissao'].isin(anos)]

# Merge das bases de dados
custeio_mt_mg_ibge = pd.merge(custeio_mt_mg, ibge_mt, how = 'left')
custeio_mt_mg_ibge_area = pd.merge(custeio_mt_mg_ibge, area_mt, how = 'left')

# Seleção das colunas
custeio_mt_mg_ibge_area_2 = custeio_mt_mg_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'Nome_Microrregião', 'AnoEmissao', 'Trimestre','AR_MUN_2022','VlCusteio']]

# Utilização das faixas de valores de custeio
custeio_mt_mg_ibge_area_2['VlCusteio'] = custeio_mt_mg_ibge_area_2['VlCusteio'].apply(func_faixa_t)

# Identificação dos nomes das mesorregiões
meso = custeio_mt_mg_ibge_area_2['Nome_Mesorregião'].unique()

# Criação de um dicionário para armazenar os resultados
resultados = {}

for nome in meso:
  print('--------------------------------------------------------------------------------')
  print(f'Análise da Mesorregiao: {nome}')

  # Escolha da mesorregião
  custeio_mt_mg_ibge_area_3 = custeio_mt_mg_ibge_area_2[custeio_mt_mg_ibge_area_2['Nome_Mesorregião'] == nome]

  # Identificação das Microrregiões de cada Mesorregião
  micro = custeio_mt_mg_ibge_area_3['Nome_Microrregião'].unique()
  print(f'Essa Mesorregiao é formada por {len(micro)} Microrregiões: {micro}')

  # Verificar soma de contratos por produtos para a mesorregião
  custeio_mt_mg_ibge_area_3_s = custeio_mt_mg_ibge_area_3.groupby(['nomeProduto'])['VlCusteio'].sum()

  # Obter os quatro principais produtos para a mesorregião
  custeio_top_products = custeio_mt_mg_ibge_area_3_s.nlargest(4).reset_index()
  print(f'Essa mesorregião tem os 4 maiores valores de custeio para os seguintes produtos: ')
  print(custeio_top_products['nomeProduto'].tolist())
  selecionados = custeio_top_products['nomeProduto'].tolist()

  # Filtrar os dados para incluir apenas os 4 produtos com maior VlCusteio
  custeio_mt_mg_prod_ibge_area_3 = custeio_mt_mg_ibge_area_3[custeio_mt_mg_ibge_area_3['nomeProduto'].isin(selecionados)]

  # Selecionar colunas
  custeio_mt_mg_prod_ibge_area_3 = custeio_mt_mg_prod_ibge_area_3[['nomeProduto', 'Nome_Microrregião', 'Trimestre','AR_MUN_2022','VlCusteio']]

  # Gera a lista de variáveis categóricas
  cols_cat_mt = custeio_mt_mg_prod_ibge_area_3.select_dtypes(include='object').columns.tolist()

  # Criação das colunas de Dummies
  data_mt = pd.get_dummies(custeio_mt_mg_prod_ibge_area_3, columns=cols_cat_mt, drop_first=True)

  # Definição das variáveis dependentes e independentes
  X, y = data_mt.drop(['VlCusteio'], axis=1), data_mt['VlCusteio']
  class_names = data_mt['VlCusteio']

  # Particiona a base de dados
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

  # Árvore de Decisão
  from sklearn.tree import DecisionTreeClassifier
  tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
  tree_custo = tree_custo.fit(X_train, y_train)
  print("")
  print("Modelo de Árvore de Decisão - DecisionTreeClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
  y_pred = tree_custo.predict(X_test)
  print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, y_pred))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, y_pred)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Naive Bayes
  from sklearn.naive_bayes import GaussianNB
  nb = GaussianNB()
  b = nb.fit(X_train, y_train)
  print("")
  print("Modelo de Naive Bayes - GaussianNB")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", nb.score(X_train, y_train))
  tp_nb = nb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_nb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_nb)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Gradiente Descendente
  from sklearn.linear_model import SGDClassifier
  sgd = SGDClassifier()
  sgd = sgd.fit(X_train, y_train)
  print("")
  print("Modelo Gradiente Descendente - SGDClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", sgd.score(X_train, y_train))
  tp_sgd = sgd.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_sgd))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_sgd)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando KNN (K - Nearest Neighbors)
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier()
  knn = knn.fit(X_train, y_train)
  print("")
  print("Modelo KNN (K - Nearest Neighbors) - KNeighborsClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", knn.score(X_train, y_train))
  tp_knn = knn.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_knn))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_knn)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Randon Forest
  from sklearn.ensemble import RandomForestClassifier
  rfm = RandomForestClassifier()
  rfm = rfm.fit(X_train, y_train)
  print("")
  print("Modelo Randon Forest - RandomForestClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", rfm.score(X_train, y_train))
  tp_rfm = rfm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_rfm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_rfm)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_mt['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo SVC
  from sklearn.svm import SVC
  svm = SVC()
  svm.fit(X_train, y_train)
  print("")
  print("Modelo SVC - Support Vector Classification")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", svm.score(X_train, y_train))
  tp_svm = svm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_svm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_svm)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # GradientBoostingClassifier
  from sklearn.ensemble import GradientBoostingClassifier
  gb = GradientBoostingClassifier()
  gb.fit(X_train, y_train)
  print("")
  print("Modelo GradientBoostingClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", gb.score(X_train, y_train))
  tp_gb = gb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_gb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_gb)
  nf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo AdaBoostClassifier
  from sklearn.ensemble import AdaBoostClassifier
  ada = AdaBoostClassifier()
  ada.fit(X_train, y_train)
  print("")
  print("Modelo AdaBoostClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", ada.score(X_train, y_train))
  tp_ada = ada.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_ada))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_ada)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo ExtraTreesClassifier
  from sklearn.ensemble import ExtraTreesClassifier
  et = ExtraTreesClassifier()
  et.fit(X_train, y_train)
  print("")
  print("Modelo ExtraTreesClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", et.score(X_train, y_train))
  tp_et = et.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_et))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_et)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Regressão Logística
  from sklearn.linear_model import LogisticRegression
  lr = LogisticRegression()
  lr.fit(X_train, y_train)
  print("")
  print("Modelo Regressão Logística")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", lr.score(X_train, y_train))
  tp_lr = lr.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_lr))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_lr)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_mt['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_mt['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Armazena os resultados em um dicionário
  resultados[nome] = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for nome, resultados_mesorregiao in resultados.items():
    print(f'Análise da Mesorregião: {nome}\n')
    df_resultados = pd.DataFrame(resultados_mesorregiao).T
    print(df_resultados)
    print('\n')



### Anual com Seleção de Produtos

In [ ]:
# Funções para Faixas de Custeio
# Anual
def func_faixa_a(x):
  if x <= 600000:
    return 1
  elif x <= 3000000:
    return 5
  else:
    return 15


# Preparação das bases
custeio_ma = custeio.copy()
ibge_ma = ibge.copy()
area_ma = area.copy()

# Soma de VlCusteio por município por ano
custeio_ma_s = custeio_ma.groupby(['Municipio', 'nomeProduto', 'AnoEmissao', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()

# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio
custeio_ma_mg = custeio_ma_s[custeio_ma_s['cdEstado'] == 12]

anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_ma_mg = custeio_ma_mg[custeio_ma_mg['AnoEmissao'].isin(anos)]

# Merge das bases de dados
custeio_ma_mg_ibge = pd.merge(custeio_ma_mg, ibge_ma, how = 'left')
custeio_ma_mg_ibge_area = pd.merge(custeio_ma_mg_ibge, area_ma, how = 'left')

# Seleção das colunas
custeio_ma_mg_ibge_area_2 = custeio_ma_mg_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'Nome_Microrregião', 'AnoEmissao','AR_MUN_2022','VlCusteio']]

# Utilização das faixas de valores de custeio
custeio_ma_mg_ibge_area_2['VlCusteio'] = custeio_ma_mg_ibge_area_2['VlCusteio'].apply(func_faixa_a)

# Identificação dos nomes das mesorregiões
meso = custeio_ma_mg_ibge_area_2['Nome_Mesorregião'].unique()

# Criação de um dicionário para armazenar os resultados
resultados = {}

for nome in meso:
  print('--------------------------------------------------------------------------------')
  print(f'Análise da Mesorregiao: ',nome)

  # Mesorregiões
  custeio_ma_mg_ibge_area_3 = custeio_ma_mg_ibge_area_2[custeio_ma_mg_ibge_area_2['Nome_Mesorregião'] == nome]

  # Identificação das Microrregiões de cada Mesorregião
  micro = custeio_ma_mg_ibge_area_3['Nome_Microrregião'].unique()
  print(f'Essa Mesorregiao é formada por {len(micro)} Microrregiões: ',micro)

  # Verificar soma de contratos por produtos para a mesorregião
  custeio_ma_mg_ibge_area_3_s = custeio_ma_mg_ibge_area_3.groupby(['nomeProduto'])['VlCusteio'].sum()

  # Obter os quatro principais produtos para a mesorregião
  custeio_top_products = custeio_ma_mg_ibge_area_3_s.nlargest(4).reset_index()
  print(f'Essa mesorregião tem os 4 maiores valores de custeio para os seguintes produtos: ')
  print(custeio_top_products['nomeProduto'].tolist())
  selecionados = custeio_top_products['nomeProduto'].tolist()

  # Filtrar os dados para incluir apenas os 10 produtos com maior VlCusteio
  custeio_ma_mg_prod_ibge_area_3 = custeio_ma_mg_ibge_area_3[custeio_ma_mg_ibge_area_3['nomeProduto'].isin(selecionados)]

  # Selecionar colunas
  custeio_ma_mg_prod_ibge_area_3 = custeio_ma_mg_prod_ibge_area_3[['nomeProduto', 'Nome_Microrregião','AR_MUN_2022','VlCusteio']]

  # Gera a lista de variáveis categóricas
  cols_cat_ma = custeio_ma_mg_prod_ibge_area_3.select_dtypes(include='object').columns.tolist()

  # Criação das colunas de Dummies
  data_ma = pd.get_dummies(custeio_ma_mg_prod_ibge_area_3, columns=cols_cat_ma, drop_first=True)

  # Definição das variáveis dependentes e independentes
  X, y = data_ma.drop(['VlCusteio'], axis=1), data_ma['VlCusteio']
  class_names = data_ma['VlCusteio']

  # Particiona a base de dados
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

  # Árvore de Decisão
  from sklearn.tree import DecisionTreeClassifier
  tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
  tree_custo = tree_custo.fit(X_train, y_train)
  print("")
  print("Modelo de Árvore de Decisão - DecisionTreeClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
  y_pred = tree_custo.predict(X_test)
  print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, y_pred))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, y_pred)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Naive Bayes
  from sklearn.naive_bayes import GaussianNB
  nb = GaussianNB()
  b = nb.fit(X_train, y_train)
  print("")
  print("Modelo de Naive Bayes - GaussianNB")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", nb.score(X_train, y_train))
  tp_nb = nb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_nb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_nb)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Gradiente Descendente
  from sklearn.linear_model import SGDClassifier
  sgd = SGDClassifier()
  sgd = sgd.fit(X_train, y_train)
  print("")
  print("Modelo Gradiente Descendente - SGDClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", sgd.score(X_train, y_train))
  tp_sgd = sgd.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_sgd))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_sgd)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando KNN (K - Nearest Neighbors)
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier()
  knn = knn.fit(X_train, y_train)
  print("")
  print("Modelo KNN (K - Nearest Neighbors) - KNeighborsClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", knn.score(X_train, y_train))
  tp_knn = knn.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_knn))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_knn)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  #Criação do modelo utilizando Randon Forest
  from sklearn.ensemble import RandomForestClassifier
  rfm = RandomForestClassifier()
  rfm = rfm.fit(X_train, y_train)
  print("")
  print("Modelo Randon Forest - RandomForestClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", rfm.score(X_train, y_train))
  tp_rfm = rfm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_rfm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_rfm)
  cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo SVC
  from sklearn.svm import SVC
  svm = SVC()
  svm.fit(X_train, y_train)
  print("")
  print("Modelo SVC - Support Vector Classification")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", svm.score(X_train, y_train))
  tp_svm = svm.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_svm))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_svm)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # GradientBoostingClassifier
  from sklearn.ensemble import GradientBoostingClassifier
  gb = GradientBoostingClassifier()
  gb.fit(X_train, y_train)
  print("")
  print("Modelo GradientBoostingClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", gb.score(X_train, y_train))
  tp_gb = gb.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_gb))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_gb)
  nf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo AdaBoostClassifier
  from sklearn.ensemble import AdaBoostClassifier
  ada = AdaBoostClassifier()
  ada.fit(X_train, y_train)
  print("")
  print("Modelo AdaBoostClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", ada.score(X_train, y_train))
  tp_ada = ada.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_ada))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_ada)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Modelo ExtraTreesClassifier
  from sklearn.ensemble import ExtraTreesClassifier
  et = ExtraTreesClassifier()
  et.fit(X_train, y_train)
  print("")
  print("Modelo ExtraTreesClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", et.score(X_train, y_train))
  tp_et = et.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_et))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_et)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Regressão Logística
  from sklearn.linear_model import LogisticRegression
  lr = LogisticRegression()
  lr.fit(X_train, y_train)
  print("")
  print("Modelo ExtraTreesClassifier")
  print("")
  print("Dados de Acurácia")
  print("Acurácia (base de treinamento): ", lr.score(X_train, y_train))
  tp_lr = lr.predict(X_test)
  print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))
  print("")
  print("Classification Report:")
  print("")
  print(classification_report(y_test, tp_lr))
  print("")
  print("Matriz de Confusão:")
  print("")
  cnf_matrix = confusion_matrix(y_test, tp_lr)
  cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
  print(cnf_table)
  print("")
  print("")

  # Armazena os resultados em um dicionário
  resultados[nome] = {
        'Árvore de Decisão': {
            'Acurácia Treinamento': tree_custo.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, y_pred)
        },
        'Naive Bayes': {
            'Acurácia Treinamento': nb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_nb)
        },
        'Gradiente Descendente': {
            'Acurácia Treinamento': sgd.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_sgd)
        },
        'KNN': {
            'Acurácia Treinamento': knn.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_knn)
        },
        'Random Forest': {
            'Acurácia Treinamento': rfm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_rfm)
        },
        'SVC': {
            'Acurácia Treinamento': svm.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_svm)
        },
        'GradientBoostingClassifier': {
            'Acurácia Treinamento': gb.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_gb)
        },
        'AdaBoostClassifier': {
            'Acurácia Treinamento': ada.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_ada)
        },
        'ExtraTreesClassifier': {
            'Acurácia Treinamento': et.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_et)
        },
        'Regressão Logística': {
            'Acurácia Treinamento': lr.score(X_train, y_train),
            'Acurácia Teste': accuracy_score(y_test, tp_lr)
        }

    }

# Criar e exibir as tabelas de resultados
for nome, resultados_mesorregiao in resultados.items():
    print(f'Análise da Mesorregião: {nome}\n')
    df_resultados = pd.DataFrame(resultados_mesorregiao).T
    print(df_resultados)
    print('\n')



## Previsões

### Previsões para modelos do tipo Trimestral (Sem 'AnoEmissao')

In [ ]:
X_train.head(2)

In [ ]:
# A última mesorregião avaliada no modelo acima foi a Norte de Minas
## 'nomeProduto': dummie ['"BOVINOS"', '"CAFÉ"', '"BANANA"', '"SOJA"']
## 'Nome_Microrregião': dummie
## 'AR_MUN_2022': número real positivo

"""
Órdem das colunas em X_train:


AR_MUN_2022


nomeProduto_"CAFÉ"
nomeProduto_"MILHO"
nomeProduto_"SOJA"


Nome_Mesorregião_Central Mineira
Nome_Mesorregião_Jequitinhonha
Nome_Mesorregião_Metropolitana de Belo Horizonte
Nome_Mesorregião_Noroeste de Minas
Nome_Mesorregião_Norte de Minas
Nome_Mesorregião_Oeste de Minas
Nome_Mesorregião_Sul/Sudoeste de Minas
Nome_Mesorregião_Triângulo Mineiro/Alto Paranaíba
Nome_Mesorregião_Vale do Mucuri
Nome_Mesorregião_Vale do Rio Doce
Nome_Mesorregião_Zona da Mata


Trimestre_quarto
Trimestre_segundo
Trimestre_terceiro

"""


# Importar bibliotecas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

v_prod = ['"BOVINOS"','"CAFÉ"','"MILHO"','"SOJA"']
v_meso = ['Campo das Vertentes','Central Mineira','Jequitinhonha','Metropolitana de Belo Horizonte','Noroeste de Minas',
           'Oeste de Minas','Sul/Sudoeste de Minas','Triângulo Mineiro/Alto Paranaíba','Vale do Mucuri',
           'Vale do Rio Doce','Zona da Mata']
v_trim = ['Primeiro', ' Quarto', 'Segundo', 'Terceiro']

valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_meso = v_meso[1]
valor_trimestre = v_trim[1]


# Árvore de Decisão
previsao_tree_custo = tree_custo.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Arvore de Decisão na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_tree_custo}\n\n')

# Naive Bayes
previsao_nb = nb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Naive Bayes na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_nb}\n\n')

# Gradiente Descendente
previsao_sgd = sgd.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Gradiente Descendente na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_sgd}\n\n')

# KNN (K - Nearest Neighbors)
previsao_knn = knn.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para K - Nearest Neighbors na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_knn}\n\n')

# Random Forest
previsao_rfm = rfm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Random Forest na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_rfm}\n\n')

# SVC (Support Vector Classification)
previsao_svm = svm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Support Vector Classification na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_svm}\n\n')

# Gradient Boosting Classifier
previsao_gb = gb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Gradient Boosting Classifier na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_gb}\n\n')

# AdaBoost Classifier
previsao_ada = ada.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para AdaBoost Classifier na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_ada}\n\n')

# Extra Trees Classifier
previsao_et = et.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Extra Trees Classifier na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_et}\n\n')

# Regressão Logística
previsao_lr = lr.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Regressão Logística na mesorregião {valor_meso}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_lr}\n\n')


In [ ]:
# Obter os nomes das colunas
colunas = X_train.columns

# Criar um dicionário para mapear os valores
valores = [4000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0]
valores_mapeados = dict(zip(colunas, valores))

# Obter as colunas correspondentes aos valores fornecidos
colunas_correspondentes = [coluna for coluna, valor in valores_mapeados.items() if valor != 0]

print(colunas_correspondentes)

### Previsões para modelos do tipo Anual

In [ ]:
X_train.head(2)

In [ ]:
# A última mesorregião avaliada no modelo acima foi a Norte de Minas
## 'nomeProduto': dummie (4)
## 'Nome_Mesorregião': dummie (12)
## 'AR_MUN_2022': número real positivo

'''
Órdem das colunas em X_train:

AR_MUN_2022

nomeProduto_"CAFÉ"
nomeProduto_"MILHO"
nomeProduto_"SOJA"

Nome_Mesorregião_Central Mineira
Nome_Mesorregião_Jequitinhonha
Nome_Mesorregião_Metropolitana de Belo Horizonte
Nome_Mesorregião_Noroeste de Minas
Nome_Mesorregião_Norte de Minas
Nome_Mesorregião_Oeste de Minas
Nome_Mesorregião_Sul/Sudoeste de Minas
Nome_Mesorregião_Triângulo Mineiro/Alto Paranaíba
Nome_Mesorregião_Vale do Mucuri
Nome_Mesorregião_Vale do Rio Doce
Nome_Mesorregião_Zona da Mata
'''

# Importar bibliotecas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

v_prod = ['"BOVINOS"','"CAFÉ"','"MILHO"','"SOJA"']
v_meso = ['Campo das Vertentes','Central Mineira','Jequitinhonha','Metropolitana de Belo Horizonte','Noroeste de Minas',
           'Norte de Minas','Oeste de Minas','Sul/Sudoeste de Minas','Triângulo Mineiro/Alto Paranaíba','Vale do Mucuri',
           'Vale do Rio Doce','Zona da Mata']

valores_para_previsao = [[400,0,1,0,0,0,0,1,0,0,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[2]
valor_meso = v_meso[4]


# Árvore de Decisão
previsao_tree_custo = tree_custo.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Arvore de Decisão na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_tree_custo}\n\n')

# Naive Bayes
previsao_nb = nb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Naive Bayes na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_nb}\n\n')

# Gradiente Descendente
previsao_sgd = sgd.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Gradiente Descendente na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_sgd}\n\n')

# KNN (K - Nearest Neighbors)
previsao_knn = knn.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para K - Nearest Neighbors na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_knn}\n\n')

# Random Forest
previsao_rfm = rfm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Random Forest na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_rfm}\n\n')

# SVC (Support Vector Classification)
previsao_svm = svm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Support Vector Classification na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_svm}\n\n')

# Gradient Boosting Classifier
previsao_gb = gb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Gradient Boosting Classifier na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_gb}\n\n')

# AdaBoost Classifier
previsao_ada = ada.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para AdaBoost Classifier na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_ada}\n\n')

# Extra Trees Classifier
previsao_et = et.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Extra Trees Classifier na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_et}\n\n')

# Regressão Logística
previsao_lr = lr.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Regressão Logística na mesorregião {valor_meso}, produto {valor_produto} e área de {valor_area} km²: {previsao_lr}\n\n')


In [ ]:
# Obter os nomes das colunas
colunas = X_train.columns

# Criar um dicionário para mapear os valores
valores = [400,0,1,0,0,0,0,1,0,0,0,0,0,0,0]
valores_mapeados = dict(zip(colunas, valores))

# Obter as colunas correspondentes aos valores fornecidos
colunas_correspondentes = [coluna for coluna, valor in valores_mapeados.items() if valor != 0]

print(colunas_correspondentes)

### Previsões para modelos mesorregionais do tipo Trimestral com seleção de produtos

In [ ]:
X_train.head(2)

In [ ]:
# A última mesorregião avaliada no modelo acima foi a Norte de Minas
## 'nomeProduto': dummie ['"BOVINOS"', '"CAFÉ"', '"BANANA"', '"SOJA"']
## 'Nome_Microrregião': dummie ['Salinas' 'Bocaiúva' 'Januária' 'Grão Mogol' 'Montes Claros' 'Pirapora'  'Janaúba']
## 'AR_MUN_2022': número real positivo

"""

Órdem das colunas em X_train:

AR_MUN_2022


nomeProduto_"BOVINOS"
nomeProduto_"CAFÉ"
nomeProduto_"SOJA"


Nome_Microrregião_Grão Mogol
Nome_Microrregião_Janaúba
Nome_Microrregião_Januária
Nome_Microrregião_Montes Claros
Nome_Microrregião_Pirapora
Nome_Microrregião_Salinas


Trimestre_quarto
Trimestre_segundo
Trimestre_terceiro
"""


# Importar bibliotecas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

v_prod = ['"BANANA"','"BOVINOS"','"CAFÉ"','"SOJA"']
v_micro = ['Bocaiúva','Grão Mogol','Janaúba','Januária','Montes Claros','Pirapora','Salinas']
v_trim = ['Primeiro', ' Quarto', 'Segundo', 'Terceiro']

valores_para_previsao = [[4000,0,1,0,0,1,0,0,0,0,0,1,0]]
valor_area = 4000
valor_produto = v_prod[2]
valor_micro = v_micro[2]
valor_trimestre = v_trim[2]
valor_meso = nome


# Árvore de Decisão
previsao_tree_custo = tree_custo.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Arvore de Decisão na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_tree_custo}\n\n')

# Naive Bayes
previsao_nb = nb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Naive Bayes na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_nb}\n\n')

# Gradiente Descendente
previsao_sgd = sgd.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Gradiente Descendente na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_sgd}\n\n')

# KNN (K - Nearest Neighbors)
previsao_knn = knn.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para K - Nearest Neighbors na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_knn}\n\n')

# Random Forest
previsao_rfm = rfm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Random Forest na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_rfm}\n\n')
# SVC (Support Vector Classification)
previsao_svm = svm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Support Vector Classification na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_svm}\n\n')

# Gradient Boosting Classifier
previsao_gb = gb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Gradient Boosting Classifier na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_gb}\n\n')

# AdaBoost Classifier
previsao_ada = ada.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para AdaBoost Classifier na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_ada}\n\n')

# Extra Trees Classifier
previsao_et = et.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Extra Trees Classifier na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_et}\n\n')

# Regressão Logística
previsao_lr = lr.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Regressão Logística na mesorregião {valor_meso}, microrregião {valor_micro}, {valor_trimestre} trimestre, produto {valor_produto} e área de {valor_area} km²: {previsao_lr}\n\n')


In [ ]:
# Obter os nomes das colunas
colunas = X_train.columns

# Criar um dicionário para mapear os valores
valores = [4000,0,1,0,0,1,0,0,0,0,0,1,0]
valores_mapeados = dict(zip(colunas, valores))

# Obter as colunas correspondentes aos valores fornecidos
colunas_correspondentes = [coluna for coluna, valor in valores_mapeados.items() if valor != 0]

print(colunas_correspondentes)

### Previsões para modelos mesorregionais do tipo Anual com seleção de produtos

In [ ]:
X_train.head(2)

In [ ]:
# A última mesorregião avaliada no modelo acima foi a Norte de Minas
## 'nomeProduto': dummie ['"BOVINOS"', '"CAFÉ"', '"BANANA"', '"SOJA"']
## 'Nome_Microrregião': dummie ['Salinas' 'Bocaiúva' 'Januária' 'Grão Mogol' 'Montes Claros' 'Pirapora'  'Janaúba']
## 'AR_MUN_2022': número real positivo

"""
Órdem das colunas em X_train:

AR_MUN_2022

nomeProduto_"BOVINOS"
nomeProduto_"CAFÉ"
nomeProduto_"SOJA"

Nome_Microrregião_Grão Mogol
Nome_Microrregião_Janaúba
Nome_Microrregião_Januária
Nome_Microrregião_Montes Claros
Nome_Microrregião_Pirapora
Nome_Microrregião_Salinas
"""


# Importar bibliotecas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

v_prod = ['"BANANA"','"BOVINOS"','"CAFÉ"','"SOJA"']
v_micro = ['Bocaiúva','Grão Mogol','Janaúba','Januária','Montes Claros','Pirapora','Salinas']
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]


# Árvore de Decisão
previsao_tree_custo = tree_custo.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Arvore de Decisão na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_tree_custo}\n\n')

# Naive Bayes
previsao_nb = nb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Naive Bayes na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_nb}\n\n')

# Gradiente Descendente
previsao_sgd = sgd.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Gradiente Descendente na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_sgd}\n\n')

# KNN (K - Nearest Neighbors)
previsao_knn = knn.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para K - Nearest Neighbors na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_knn}\n\n')

# Random Forest
previsao_rfm = rfm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Random Forest na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_rfm}\n\n')

# SVC (Support Vector Classification)
previsao_svm = svm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Support Vector Classification na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_svm}\n\n')

# Gradient Boosting Classifier
previsao_gb = gb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Gradient Boosting Classifier na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_gb}\n\n')

# AdaBoost Classifier
previsao_ada = ada.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para AdaBoost Classifier na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_ada}\n\n')

# Extra Trees Classifier
previsao_et = et.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Extra Trees Classifier na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_et}\n\n')

# Regressão Logística
previsao_lr = lr.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Regressão Logística na mesorregião {nome}, microrregião {valor_micro}, produto {valor_produto} e área de {valor_area} km²: {previsao_lr}\n\n')


In [ ]:
# Obter os nomes das colunas
colunas = X_train.columns

# Criar um dicionário para mapear os valores
valores = [400,1,0,0,1,0,0,0,0,0]
valores_mapeados = dict(zip(colunas, valores))

# Obter as colunas correspondentes aos valores fornecidos
colunas_correspondentes = [coluna for coluna, valor in valores_mapeados.items() if valor != 0]

print(colunas_correspondentes)

## Mesorregião Triângulo Mineiro/Alto Paranaíba

In [ ]:
from sklearn.model_selection import cross_val_score
# Funções para Faixas de Custeio
# Anual
def func_faixa_a(x):
  if x <= 600000:
    return 1
  elif x <= 3000000:
    return 5
  else:
    return 15


# Preparação das bases
custeio_ma = custeio.copy()
ibge_ma = ibge.copy()
area_ma = area.copy()

# Soma de VlCusteio por município por ano
custeio_ma_s = custeio_ma.groupby(['Municipio', 'nomeProduto', 'AnoEmissao', 'cdEstado', 'codIbge'], as_index=False)['VlCusteio'].sum()

# Escolha do estado (MG = 12), seleção dos objetos e filtragem da base custeio
custeio_ma_mg = custeio_ma_s[custeio_ma_s['cdEstado'] == 12]

anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
custeio_ma_mg = custeio_ma_mg[custeio_ma_mg['AnoEmissao'].isin(anos)]

# Merge das bases de dados
custeio_ma_mg_ibge = pd.merge(custeio_ma_mg, ibge_ma, how = 'left')
custeio_ma_mg_ibge_area = pd.merge(custeio_ma_mg_ibge, area_ma, how = 'left')

# Seleção das colunas
custeio_ma_mg_ibge_area_2 = custeio_ma_mg_ibge_area[['nomeProduto',	'Nome_Mesorregião', 'Nome_Microrregião', 'AnoEmissao','AR_MUN_2022','VlCusteio']]

# Utilização das faixas de valores de custeio
custeio_ma_mg_ibge_area_2['VlCusteio'] = custeio_ma_mg_ibge_area_2['VlCusteio'].apply(func_faixa_a)

# Mesorregião Triângulo Mineiro/Alto Paranaíba
custeio_ma_mg_ibge_area_2 = custeio_ma_mg_ibge_area_2[custeio_ma_mg_ibge_area_2['Nome_Mesorregião'] == 'Triângulo Mineiro/Alto Paranaíba']

# Criação de um dicionário para armazenar os resultados
resultados = {}

print('--------------------------------------------------------------------------------')
print('Análise da Mesorregiao Triângulo Mineiro/Alto Paranaíba')

# Identificação das Microrregiões de cada Mesorregião
micro = custeio_ma_mg_ibge_area_2['Nome_Microrregião'].unique()
print(f'Essa Mesorregiao é formada por {len(micro)} Microrregiões: ',micro)

# Verificar soma de contratos por produtos para a mesorregião
custeio_ma_mg_ibge_area_2_s = custeio_ma_mg_ibge_area_2.groupby(['nomeProduto'])['VlCusteio'].sum()

# Obter os quatro principais produtos para a mesorregião
custeio_top_products = custeio_ma_mg_ibge_area_2_s.nlargest(4).reset_index()
print(f'Essa mesorregião tem os 4 maiores valores de custeio para os seguintes produtos: ')
print(custeio_top_products['nomeProduto'].tolist())
selecionados = custeio_top_products['nomeProduto'].tolist()

# Filtrar os dados para incluir apenas os 10 produtos com maior VlCusteio
custeio_ma_mg_prod_ibge_area_2 = custeio_ma_mg_ibge_area_2[custeio_ma_mg_ibge_area_2['nomeProduto'].isin(selecionados)]

# Selecionar colunas
custeio_ma_mg_prod_ibge_area_2 = custeio_ma_mg_prod_ibge_area_2[['nomeProduto', 'Nome_Microrregião','AR_MUN_2022','VlCusteio']]

# Gera a lista de variáveis categóricas
cols_cat_ma = custeio_ma_mg_prod_ibge_area_2.select_dtypes(include='object').columns.tolist()

# Criação das colunas de Dummies
data_ma = pd.get_dummies(custeio_ma_mg_prod_ibge_area_2, columns=cols_cat_ma, drop_first=True)

# Definição das variáveis dependentes e independentes
X, y = data_ma.drop(['VlCusteio'], axis=1), data_ma['VlCusteio']
class_names = data_ma['VlCusteio']

# Particiona a base de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)




In [ ]:
# Árvore de Decisão
from sklearn.tree import DecisionTreeClassifier
tree_custo = DecisionTreeClassifier(random_state=0, criterion='log_loss')
tree_custo = tree_custo.fit(X_train, y_train)
print("")
print("Modelo de Árvore de Decisão - DecisionTreeClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento):", tree_custo.score(X_train, y_train))
y_pred = tree_custo.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, y_pred))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(tree_custo, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Árvore de Decisão

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]


previsao_tree_custo = tree_custo.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Arvore de Decisão na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_tree_custo}\n\n')



In [ ]:
# Árvore de Decisão

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[10000,0,0,0,0,1,0,0,0,0]]
valor_area = 10000
valor_produto = v_prod[0]
valor_micro = v_micro[2]


previsao_tree_custo = tree_custo.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Arvore de Decisão na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_tree_custo}\n\n')



In [ ]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb = nb.fit(X_train, y_train)
print("")
print("Modelo de Naive Bayes - GaussianNB")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", nb.score(X_train, y_train))
tp_nb = nb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_nb))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_nb))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(nb, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_nb)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Naive Bayes

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]


previsao_nb = nb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Naive Bayes na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_nb}\n\n')



In [ ]:
#Criação do modelo utilizando Gradiente Descendente
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd = sgd.fit(X_train, y_train)
print("")
print("Modelo Gradiente Descendente - SGDClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", sgd.score(X_train, y_train))
tp_sgd = sgd.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_sgd))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_sgd))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(sgd, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_sgd)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Gradiente Descendente

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_sgd = sgd.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Gradiente Descendente na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_sgd}\n\n')



In [ ]:
#Criação do modelo utilizando KNN (K - Nearest Neighbors)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn = knn.fit(X_train, y_train)
print("")
print("Modelo KNN (K - Nearest Neighbors) - KNeighborsClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", knn.score(X_train, y_train))
tp_knn = knn.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_knn))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_knn))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(knn, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_knn)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# KNN (K - Nearest Neighbors)

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_knn = knn.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para K - Nearest Neighbors na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_knn}\n\n')



In [ ]:
#Criação do modelo utilizando Randon Forest
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm = rfm.fit(X_train, y_train)
print("")
print("Modelo Randon Forest - RandomForestClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", rfm.score(X_train, y_train))
tp_rfm = rfm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_rfm))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_rfm))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(rfm, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_rfm)
cnf_table = pd.DataFrame(data = cnf_matrix, index = data_ma['VlCusteio'].unique(), columns = [str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Random Forest

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_rfm = rfm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Random Forest na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_rfm}\n\n')



In [ ]:
# Modelo SVC
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print("")
print("Modelo SVC - Support Vector Classification")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", svm.score(X_train, y_train))
tp_svm = svm.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_svm))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_svm))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(svm, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_svm)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# SVC (Support Vector Classification)

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_svm = svm.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio anual para Support Vector Classification na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_svm}\n\n')



In [ ]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
print("")
print("Modelo GradientBoostingClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", gb.score(X_train, y_train))
tp_gb = gb.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_gb))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_gb))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(gb, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_gb)
nf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Gradient Boosting Classifier

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_gb = gb.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Gradient Boosting Classifier na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_gb}\n\n')



In [ ]:
# Modelo AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
print("")
print("Modelo AdaBoostClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", ada.score(X_train, y_train))
tp_ada = ada.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_ada))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_ada))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(ada, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_ada)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# AdaBoost Classifier

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_ada = ada.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para AdaBoost Classifier na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_ada}\n\n')



In [ ]:
# Modelo ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier()
et.fit(X_train, y_train)
print("")
print("Modelo ExtraTreesClassifier")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", et.score(X_train, y_train))
tp_et = et.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_et))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_et))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(et, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_et)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")



In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Extra Trees Classifier

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_et = et.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Extra Trees Classifier na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_et}\n\n')



In [ ]:
# Regressão Logística
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
print("")
print("Modelo LogisticRegression")
print("")
print("Dados de Acurácia")
print("Acurácia (base de treinamento): ", lr.score(X_train, y_train))
tp_lr = lr.predict(X_test)
print("Acurácia de previsão: ", accuracy_score(y_test, tp_lr))
print("")
print("Classification Report:")
print("")
print(classification_report(y_test, tp_lr))
print("")
print("Validação Cruzada:")
print("")
scores = cross_val_score(lr, X, y, cv=5)
score = scores.sum()/5
print("Cross-validation: ",scores)
print("Média cross-validation: ", score)
print("")
print("Matriz de Confusão:")
print("")
cnf_matrix = confusion_matrix(y_test, tp_lr)
cnf_table = pd.DataFrame(data=cnf_matrix, index=data_ma['VlCusteio'].unique(), columns=[str(x) + '_prev' for x in data_ma['VlCusteio'].unique()])
print(cnf_table)
print("")
print("")





In [ ]:
# Plotar mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(cnf_table, annot=True, cmap="YlGnBu")
plt.xlabel('Valor Previsto')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
# Regressão Logística

v_prod = selecionados
v_micro = micro
valores_para_previsao = [[4000,1,0,0,1,0,0,0,0,0]]
valor_area = 4000
valor_produto = v_prod[1]
valor_micro = v_micro[1]

previsao_lr = lr.predict(valores_para_previsao)
print(f'Previsão de faixa de custeio trimestral para Regressão Logística na mesorregião Triângulo Mineiro / Alto Paranaíba, microrregião {valor_micro} produto {valor_produto} e área de {valor_area} km²: {previsao_lr}\n\n')


In [ ]:
# Armazena os resultados em um dicionário
resultados = {
    'Árvore de Decisão': {
        'Acurácia Treinamento': [tree_custo.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, y_pred)],
        'Cross-Validation': [cross_val_score(tree_custo, X, y, cv=5).mean()]
    },
    'Naive Bayes': {
        'Acurácia Treinamento': [nb.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_nb)],
        'Cross-Validation': [cross_val_score(nb, X, y, cv=5).mean()]
    },
    'Gradiente Descendente': {
        'Acurácia Treinamento': [sgd.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_sgd)],
        'Cross-Validation': [cross_val_score(sgd, X, y, cv=5).mean()]
    },
    'KNN': {
        'Acurácia Treinamento': [knn.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_knn)],
        'Cross-Validation': [cross_val_score(knn, X, y, cv=5).mean()]
    },
    'Random Forest': {
        'Acurácia Treinamento': [rfm.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_rfm)],
        'Cross-Validation': [cross_val_score(rfm, X, y, cv=5).mean()]
    },
    'SVC': {
        'Acurácia Treinamento': [svm.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_svm)],
        'Cross-Validation': [cross_val_score(svm, X, y, cv=5).mean()]
    },
    'GradientBoostingClassifier': {
        'Acurácia Treinamento': [gb.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_gb)],
        'Cross-Validation': [cross_val_score(gb, X, y, cv=5).mean()]
    },
    'AdaBoostClassifier': {
        'Acurácia Treinamento': [ada.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_ada)],
        'Cross-Validation': [cross_val_score(ada, X, y, cv=5).mean()]
    },
    'ExtraTreesClassifier': {
        'Acurácia Treinamento': [et.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_et)],
        'Cross-Validation': [cross_val_score(et, X, y, cv=5).mean()]
    },
    'Regressão Logística': {
        'Acurácia Treinamento': [lr.score(X_train, y_train)],
        'Acurácia Teste': [accuracy_score(y_test, tp_lr)],
        'Cross-Validation': [cross_val_score(lr, X, y, cv=5).mean()]
    }
}

# Criar e exibir as tabelas de resultados
for modelo, resultados_mesorregiao in resultados.items():
    df_resultados = pd.DataFrame(resultados_mesorregiao)
    df_resultados.index.name = None
    print(f"Resultados do Modelo: {modelo}\n")
    print(df_resultados)
    print('\n')
